In [23]:
import cifar_gpu_fun
reload(cifar_gpu_fun)
from cifar_gpu_fun import *
from numba import jit
import bcd
import sys
from sklearn.cluster import KMeans
from sklearn import metrics
import pyclust
import matlab.engine
import scipy.io
import pandas as pd
from sklearn.linear_model import SGDClassifier
%pylab inline

#### Define Some Constants 

In [275]:
NUM_FEATURE_BATCHES=1
DATA_BATCH_SIZE=(1280)
FEATURE_BATCH_SIZE=(1024)
NUM_TRAIN = 50000
NUM_TEST = 10000
NUM_CLASSES = 10
NUM_FEATURES = FEATURE_BATCH_SIZE * NUM_FEATURE_BATCHES * 9 * 2
POOL_TYPE ='avg'
FILTER_GEN ='gaussians'
BANDWIDTH = 1.0
LAMBDAS = [0.01, 0.1, 1.0]
CUDA_CONVNET = True
SCALE = 55.0
BIAS = 1.25
MIN_VAR_TOL = 1e-4
TOT_FEAT = FEATURE_BATCH_SIZE*NUM_FEATURE_BATCHES
NAN = float('nan')
saved = 0


In [10]:
np.random.seed(10)                                                                                                                                                                   
(XTrain, labelsTrain), (XTest, labelsTest) = load_cifar_processed()
patches = patchify_all_imgs(XTrain, (6,6), pad=False)

(50000, 3, 32, 32)


In [11]:
notebook_stdout = sys.stdout
notebook_stderr = sys.stderr
terminal_stdout = open('/dev/stdout', 'w')
terminal_stderr = open('/dev/stderr', 'w')

In [12]:
patches_flattened = patches.reshape(patches.shape[0]*patches.shape[1], *patches.shape[2:])

In [13]:
np.random.seed(0)
idxs = np.random.choice(patches_flattened.shape[0], 1024, replace=False)
sampled = patches_flattened[idxs, :]

In [181]:
VALID_POOLS = [(15,6)]

In [15]:
results = {}

In [450]:
sys.stdout = terminal_stdout
sys.stderr = terminal_stderr
pool_size, pool_stride = VALID_POOLS[0]
filter_gen = make_empirical_filter_gen(patches, labelsTrain, MIN_VAR_TOL=MIN_VAR_TOL, seed=3)
X = np.vstack((XTrain, XTest))
(X_lift, filters) = conv(X, filter_gen, 1024, 4, DATA_BATCH_SIZE, CUDA_CONVNET, pool_size=pool_size, pool_stride=pool_stride, symmetric_relu=True)
X_lift_train = X_lift[:50000,:,:,:]
X_lift_test = X_lift[50000:,:,:,:] 
X_lift_train = X_lift_train.reshape(X_lift_train.shape[0], -1)
X_lift_test = X_lift_test.reshape(X_lift_test.shape[0], -1)

In [489]:
sys.stdout = terminal_stdout
sys.stderr = terminal_stderr
pool_size, pool_stride = VALID_POOLS[0]
filter_gen = make_empirical_filter_gen(patches, labelsTrain, MIN_VAR_TOL=MIN_VAR_TOL, seed=3)
X = np.vstack((XTrain, XTest))
(X_lift, filters) = conv(X, filter_gen, 1024, 1, DATA_BATCH_SIZE, CUDA_CONVNET, pool_size=pool_size, pool_stride=pool_stride, symmetric_relu=True)
X_lift_train_small = X_lift[:50000,:,:,:]
X_lift_test_small = X_lift[50000:,:,:,:] 
X_lift_train_small = X_lift_train_small.reshape(X_lift_train.shape[0], -1)
X_lift_test_small = X_lift_test_small.reshape(X_lift_test.shape[0], -1)

In [490]:
X_lift_train_small.shape

(50000, 18432)

In [78]:
def reduce_random_features(Phi, labels, cls=0, eps=1e-8, rho=800):
    rho = float(rho)
    y = np.zeros(labels.shape[0])
    y[np.where(labels == cls)] = 1
    y[np.where(labels != cls)] = -1
    Ks = Phi.T.dot(y)[:, np.newaxis]
    Ks = Ks * Ks
    nw = Ks.shape[0]
    alphas = linear_chi_square(-1 * Ks, 1/nw * np.ones((nw,1)), rho/nw, 0.001).ravel() 
    bm = alphas > eps
    phi_compress = Phi[:, np.where(bm)].reshape(Phi.shape[0], -1)
    alphas = alphas[np.where(bm)]
    return phi_compress, np.where(bm), alphas

def linear_chi_square(ks, u, rho, tol):
    scipy.io.savemat("/tmp/ks", {"v": ks, "u": u})
    alphay = eng.linear_chi_square("/tmp/ks", rho, tol)
    return np.array(alphay)



In [473]:
idxs = np.random.choice(X_lift_train.shape[0], 5000, replace=False)
X_lift_sample = X_lift_train[idxs, :]
labels_sample = labelsTrain[idxs]
K_sample = X_lift_sample.dot(X_lift_sample.T)
w = X_lift_sample.T.dot(learnDual(K_sample, labels_sample)).T
norms = np.linalg.norm(w, axis=0)
norms_sort = np.argsort(norms)
top_features = norms_sort[-576*32:]
X_lift_train_prune = X_lift_train[:, top_features]
X_lift_test_prune = X_lift_test[:, top_features]




Learning Dual Model better
REG IS  0.1


In [452]:
X_lift_train_prune.shape

(50000, 4608)

In [494]:
sys.stdout = notebook_stdout
sys.stderr = notebook_stderr

In [485]:
train_acc_prune, test_acc_prune = trainAndEvaluatePrimalModel(X_lift_train_prune, X_lift_test_prune, labelsTrain, labelsTest, reg=100)

X SHAPE  (50000, 18432)
Computing XTX
Done Computing XTX


In [492]:
train_acc_control, test_acc_control = trainAndEvaluatePrimalModel(X_lift_train_small, X_lift_test_small, labelsTrain, labelsTest, reg=100)

In [497]:
print(train_acc_prune, train_acc_control)
print(test_acc_prune, test_acc_control)

(0.92769999999999997, 0.88959999999999995)
(0.78680000000000005, 0.77600000000000002)


In [438]:
X_lift_train_small.shape

(50000, 4608)

In [393]:
train_acc, test_acc = trainAndEvaluatePrimalModel(X_lift_train, X_lift_test, labelsTrain, labelsTest, reg=0.1*TOT_FEAT)

KeyboardInterrupt: 

In [177]:
train_acc_prune, test_acc_prune = trainAndEvaluatePrimalModel(X_lift_train_prune, X_lift_test_prune, labelsTrain, labelsTest, reg=0.0001*TOT_FEAT)

In [488]:
X_lift_train_prune.shape

(50000, 18432)

In [180]:
train_acc_prune

0.81935999999999998

In [13]:
eng = matlab.engine.start_matlab()

In [79]:
duchi_results = []
train_features_dict = {}
test_features_dict = {}
alphas_dict = {}
for rho in [10, 20, 50.0]:
    train_features = [] 
    test_features = []
    alphas = []
    for c in range(10):
        print("Compressing Class {0}".format(c))
        train_features_compress, idx, alpha = reduce_random_features(X_lift_train, labelsTrain, rho=rho, cls=c) 
        test_features_compress = X_lift_test[:, idx] 
        train_features.append(train_features_compress)
        test_features.append(test_features_compress)
        alphas.append(alpha)
    alphas_dict[rho] = alphas
    train_features_dict[rho] = train_features
    test_features_dict[rho] = test_features
    

(50000, 1278)

In [73]:
for rho in [10, 20, 50.0]:
    train_features = train_features_dict[rho]
    test_features = test_features_dict[rho]
    for reg in LAMBDAS:
        models = one_vs_all_solve(train_features, labelsTrain, 10, reg=1e-8)
        result = {}
        result['reg'] = reg
        train_acc, test_acc = one_vs_all_evaluate(train_features, test_features, labelsTrain, labelsTest, models, 10)
        result['train_acc'] = train_acc
        result['test_acc'] = test_acc
        result['rho'] = rho
        results['reduced'] = True
        result['mean_num_features'] = np.mean(map(lambda x: x.shape[1], train_features))
        duchi_results.append(result)

In [74]:
pd.DataFrame(duchi_results)

,mean_num_features,reg,rho,test_acc,train_acc
0,5395.3,0.01,10.0,0.1368,0.6597
1,5395.3,0.10,10.0,0.1368,0.6597
2,5395.3,1.00,10.0,0.1368,0.6597
3,2996.8,0.01,20.0,0.1081,0.7706
4,2996.8,0.10,20.0,0.1081,0.7706
5,2996.8,1.00,20.0,0.1081,0.7706
6,1268.5,0.01,50.0,0.1209,0.6954
7,1268.5,0.10,50.0,0.1209,0.6954
8,1268.5,1.00,50.0,0.1209,0.6954


In [15]:
def one_vs_all_solve(train_features, labels, num_classes, reg=0.1):
    models = []
    for c in range(num_classes):
        print("Solving class {0}".format(c))
        labels_c = (labels == c).astype('int')
        model = learnPrimal(train_features[c], labels_c, reg=reg)
        models.append(model)
        
    return models

def one_vs_all_predict(features, models, num_classes):
    preds = [] 
    for c in range(num_classes):
        print("Predicting class {0}".format(c))
        yHat, yRaw = evaluatePrimalModel(features[c], models[c])
        preds.append(yRaw[:, 1])
    return np.vstack(preds).T


def one_vs_all_evaluate(train_features, test_features, train_labels, test_labels, models, num_classes):
    train_preds = one_vs_all_predict(train_features, models, num_classes)
    test_preds = one_vs_all_predict(test_features, models, num_classes)
    train_pred_labels = np.argmax(train_preds, axis=1)
    test_pred_labels = np.argmax(test_preds, axis=1)
    
    train_acc = metrics.accuracy_score(train_pred_labels, train_labels)
    test_acc = metrics.accuracy_score(test_pred_labels, test_labels)
    return train_acc, test_acc

        

In [85]:
pd.DataFrame(duchi_results)

,mean_num_features,reg,rho,test_acc,train_acc
0,10049.8,0.01,5.0,0.1801,0.22760
1,10049.8,0.10,5.0,0.1801,0.22760
2,10049.8,1.00,5.0,0.1801,0.22760
3,5395.3,0.01,10.0,0.5176,0.63364
4,5395.3,0.10,10.0,0.5176,0.63364
5,5395.3,1.00,10.0,0.5176,0.63364
6,3884.9,0.01,15.0,0.6995,0.80210
7,3884.9,0.10,15.0,0.6995,0.80210
8,3884.9,1.00,15.0,0.6995,0.80210
9,2996.8,0.01,20.0,0.6927,0.77408


In [79]:
results_df

NameError: name 'results_df' is not defined

In [212]:
train_features

[]

In [194]:
models = one_vs_all_solve(train_features, labelsTrain, 10, reg=1e-8)
print(one_vs_all_evaluate(train_features, test_features, labelsTrain, labelsTest, models, 10))

Solving class 0
X SHAPE  (50000, 851)
Computing XTX
Done Computing XTX
Solving class 1
X SHAPE  (50000, 842)
Computing XTX
Done Computing XTX
Solving class 2
X SHAPE  (50000, 840)
Computing XTX
Done Computing XTX
Solving class 3
X SHAPE  (50000, 822)
Computing XTX
Done Computing XTX
Solving class 4
X SHAPE  (50000, 825)
Computing XTX
Done Computing XTX
Solving class 5
X SHAPE  (50000, 824)
Computing XTX
Done Computing XTX
Solving class 6
X SHAPE  (50000, 857)
Computing XTX
Done Computing XTX
Solving class 7
X SHAPE  (50000, 834)
Computing XTX
Done Computing XTX
Solving class 8
X SHAPE  (50000, 837)
Computing XTX
Done Computing XTX
Solving class 9
X SHAPE  (50000, 832)
Computing XTX
Done Computing XTX
Predicting class 0
Predicting class 1
Predicting class 2
Predicting class 3
Predicting class 4
Predicting class 5
Predicting class 6
Predicting class 7
Predicting class 8
Predicting class 9
Predicting class 0
Predicting class 1
Predicting class 2
Predicting class 3
Predicting class 4
Predi

In [189]:
preds = one_vs_all_predict(train_features, models, num_classes=10)

Predicting class 0
Predicting class 1
Predicting class 2
Predicting class 3
Predicting class 4
Predicting class 5
Predicting class 6
Predicting class 7
Predicting class 8
Predicting class 9


In [192]:
print(one_vs_all_evaluate(train_features, test_features, labelsTrain, labelsTest, models, 10))

Predicting class 0
Predicting class 1
Predicting class 2
Predicting class 3
Predicting class 4
Predicting class 5
Predicting class 6
Predicting class 7
Predicting class 8
Predicting class 9
Predicting class 0
Predicting class 1
Predicting class 2
Predicting class 3
Predicting class 4
Predicting class 5
Predicting class 6
Predicting class 7
Predicting class 8
Predicting class 9
(0.66720000000000002, 0.64410000000000001)


In [193]:
trainAndEvaluatePrimalModel(X_lift_train, X_lift_test, labelsTrain, labelsTest)

X SHAPE  (50000, 16384)
Computing XTX
Done Computing XTX


(0.97097999999999995, 0.7288)

In [70]:
X_lift_train = X_lift_train.reshape(X_lift_train.shape[0], -1)

In [72]:
X_lift_train.shape

(50000, 16384)

In [73]:
def convert_labels(labels, num_classes=10):
    return np.eye(num_classes)[labels]

In [77]:
X_lift_train.shape

(50000, 16384)

In [89]:
labelsTrain.shape

(50000,)

In [89]:
numpy.savez("cifar_big.npz", X_lift_train=X_lift_train, y_train=labelsTrain, y_test=labelsTest, X_lift_test=X_lift_test)

In [ ]:
y